# 第08讲 Pandas的SettingWithCopyWarning报警

1. [原因分析](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy)：
    * **链式操作（多级筛选、链式赋值等）**需要按序执行，即先执行步骤一，再在步骤一结果的基础上执行步骤二，以此类推；
    * **链式操作具有不可预测性**，即前驱步骤的执行结果不一定符合后继步骤的输入条件；
    * **链式赋值**时，需要先筛选出子DataFrame，再对子DataFrame进行赋值；
    * 无法自动判断筛选出的子DataFrame是对象的引用还是拷贝，导致赋值能否成功未知。


2. 处理思路：
    * 避免任何形式的链式赋值；
    * 将链式操作转换成一次操作；
    * 赋值操作仅在原对象上进行，并且一步到位；
    * 需要多级选取时，使用 `df.loc[:, ('one', 'second')]` 替代 `df['one']['second']`；
    * 需要对象拷贝时，使用 `df.copy()`。


3. 解决方案：
    * 方案一：使用`df.loc`一个步骤直接修改原对象
    * 方案二：使用`df.copy()`显式拷贝筛选结果，再执行赋值


4. 数据集：2018年北京市天气情况

In [1]:
import pandas as pd

## 0. 数据准备

In [2]:
fpath = "./datas/beijing_weather/beijing_weather_2018.csv"
df = pd.read_csv(fpath)

In [3]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
0,2018-01-01,3℃,-6℃,晴~多云,东北风,1-2级,59,良,2
1,2018-01-02,2℃,-5℃,阴~多云,东北风,1-2级,49,优,1
2,2018-01-03,2℃,-5℃,多云,北风,1-2级,28,优,1
3,2018-01-04,0℃,-8℃,阴,东北风,1-2级,28,优,1
4,2018-01-05,3℃,-6℃,多云~晴,西北风,1-2级,50,优,1


In [4]:
df.loc[:, "bWendu"] = df["bWendu"].str.replace("℃", "").astype('int64')
df.loc[:, "yWendu"] = df["yWendu"].str.replace("℃", "").astype('int64')

In [5]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1


## 1. 问题复现

In [6]:
condition = df["ymd"].str.startswith("2018-03")

In [7]:
df[condition]["wen_cha"] = df["bWendu"]-df["yWendu"]

<ipython-input-7-37b6a90bb5a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[condition]["wen_cha"] = df["bWendu"]-df["yWendu"]


In [8]:
df[condition].head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
59,2018-03-01,8,-3,多云,西南风,1-2级,46,优,1
60,2018-03-02,9,-1,晴~多云,北风,1-2级,95,良,2
61,2018-03-03,13,3,多云~阴,北风,1-2级,214,重度污染,5
62,2018-03-04,7,-2,阴~多云,东南风,1-2级,144,轻度污染,3
63,2018-03-05,8,-3,晴,南风,1-2级,94,良,2


## 2. 解决方案

### 2.1 方案一：一次操作



In [9]:
df.loc[condition, "wen_cha"] = df["bWendu"]-df["yWendu"]

In [10]:
df[condition].head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wen_cha
59,2018-03-01,8,-3,多云,西南风,1-2级,46,优,1,11.0
60,2018-03-02,9,-1,晴~多云,北风,1-2级,95,良,2,10.0
61,2018-03-03,13,3,多云~阴,北风,1-2级,214,重度污染,5,10.0
62,2018-03-04,7,-2,阴~多云,东南风,1-2级,144,轻度污染,3,9.0
63,2018-03-05,8,-3,晴,南风,1-2级,94,良,2,11.0


### 2.2 方案二：显式拷贝

使用`df.copy()`显式拷贝筛选结果，再执行赋值

In [11]:
df_month3 = df[condition].copy()

In [12]:
df_month3.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wen_cha
59,2018-03-01,8,-3,多云,西南风,1-2级,46,优,1,11.0
60,2018-03-02,9,-1,晴~多云,北风,1-2级,95,良,2,10.0
61,2018-03-03,13,3,多云~阴,北风,1-2级,214,重度污染,5,10.0
62,2018-03-04,7,-2,阴~多云,东南风,1-2级,144,轻度污染,3,9.0
63,2018-03-05,8,-3,晴,南风,1-2级,94,良,2,11.0


In [13]:
df_month3["wen_cha"] = df["bWendu"]-df["yWendu"]

In [14]:
df_month3.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wen_cha
59,2018-03-01,8,-3,多云,西南风,1-2级,46,优,1,11
60,2018-03-02,9,-1,晴~多云,北风,1-2级,95,良,2,10
61,2018-03-03,13,3,多云~阴,北风,1-2级,214,重度污染,5,10
62,2018-03-04,7,-2,阴~多云,东南风,1-2级,144,轻度污染,3,9
63,2018-03-05,8,-3,晴,南风,1-2级,94,良,2,11
